# Services

## Introduction

La notion de **service** est essentielle dans toutes les architectures logicielles modernes. 

> **Note** En allant au bout de la logique, certaines applications sont décomposées en multiples services indépendants appelés « _microservices_ », qui peuvent être hébergés sur des serveurs arbitraires, comme les services « _en nuages_ » (cloud computing).

Les services sont les classes qui contiennent toute la logique « _métier_ » de l'application.

La structure de base, qui comprend (avec Laravel) les **modèles**, les **contrôleurs**, les **vues**, etc. est en charge pruincipalement de la **gestion** des données et des requêtes de l'utilisateur. Ce sont souvent des fonctionnalités qui se ressemblent d'une appication à l'autre. Ce sont des services qui sont véritablement la **chair** spécifique du domaine.

L'architecture **MVC** (ou une équivalente) n'est donc pas vraiment suffisante pour représenter une application complète.

Dans les applications basées sur les services, il y a deux notions extrêmement importantes à garder en tête et sur lesquelles nous reviendrons :
- la notion de **conteneur de services**
- la notion (collatérale) d'**injection de dépendance**

## Qu'est-ce qu'un service ?

En termes logiciels, un service est un module/paquetage qui implémente des fonctionnalités propres à une partie cohérente de l'application, en lien avec unepartie cohérente des métiers qu'automatise l'application.

Par exemple, dans une application de commerce en ligne, la question de la livraison pourrait constituer un service, prenant en charge le calcul des frais de livraison, le choix d'un livreur, la vérification de la livraison, etc.

Naturellement, comme dans toute architecture, les forntières sont souvent arbitraires et on peut vouloir consttuer des services plus ou moins « _gros_ ».

En `PHP`, un service est donc principalement une classe qui implémente les fonctions requises (p. ex. une mathode pour calculer les frais de livraison en fonction de la localisation du client). Rien de plus. Souvent, on regroupera les services apparentés en paquetages de manière à constituer des classes qui ont une responsabilité limitée, conformément aux bonnes pratiques.

Voici une classe de service extrêmement simpliste :

In [1]:
<?php
namespace App\Services;

final class Foo
{
    public function lazy (int $x): int
    {
        return $x;
    }
}

SyntaxError: invalid syntax (2795157088.py, line 1)

### Injection de dépendance

Dans les architectures basées sur les services, on utilise un « **modèle de conception** » appelé **injection de dépendance**.

L'injection de dépendance consiste principalement à déclarer qu'une classe dépend d'une autre classe en déclarant cette dernière comme paramètre du constructeur.

Imaginons qu'une classe **A** ait besoin des fonctionnalités de la classe **Foo** (donc _dépende_ de cette classe), nous aurons donc juste besoin d'écrire :

In [ ]:
%%php

namespace App\Random;

use App\Services\Foo;

final class A
{
    public function __construct (
        private Foo $foo
    )
    {
    }
    
    public function f()
    {
        return $this->foo->lazy(5);
    }
}

Par la suite, il nous suffit d'écrire l'appel suivant :

In [ ]:
%%php

$a = new A(newFoo());
$a->f();

Nous avons déjà gagné en flexibilité par rapport à la situation où la dépendance serai déclarée **dans** le constructeur. D'ailleurs, pour être tout à fait correct, la classe `A` devrait déclarer :

In [ ]:
%%php

public function __construct (private FooInterface $foo) { /* ...*/}

et la classe `Foo` :

In [2]:
final class Foo implements FooInterface {/* ... */}

SyntaxError: invalid syntax (4098221863.py, line 1)

Néanmoins, nous allons devoir faire face à un problème dès que la chaîne des dépendances va s'allonger. Nous allons devoir instancier de nombreuses classes.

De surcroît, si nous utilisons des bibliothèques tierces, nous ne saurons pas toujours quelles sont les dépendances ces classes que nous utilisons.

Enfin... Avec des environnements comme Laravel ou Symfony, notre code instanciera assez rarement explicitement des classes de services (c'est d'ailleurs désormais considéré une mauuvaise pratique).

Donc, comment faire ?

### Le conteneur de services

Pour résoudre ces problèmes, Symfony ou Laravel on recours à un objet qui répertorie tous les services et qui, pour cette raison, est appelé **conteneur de services**.

Ce conteneur est contrauit au lancement de l'application.

Il scanne toutes les classes de l'application et examine leur constructeur pour savoir de quelles autres classes elles dépendent. Et ceci récursivement.

A la fin du processus, le conteneur contient un instance unique de chaque service (un singleton) qui est prêt à être appelé par l'application.

> **Note** En réalité, le processus est un peu plus complexe car le compilateur du conteneur utilise diverses stratégies pour optimiser celui-ci.

Ainsi, nous n'aurons plus jamais à nous préoccuper des sépendances des services que nous appelons. Ils sont toujours pré-configurés par l'nevironnement lui-même.

### Cas particulier des contrôleurs

Comme nous l'avons dit, le cas général est d'injecter les dépendances _par le constructeur_.

Les classes de contrôleurs ont une spécificité à cet égard puisque chaque contrôleur peut déclarer des dépendances. Ceci paraît assez logique car chaque contrôleur est indépendant des autres, même au sein d'une même classe.

Typiquement, on verra donc ce genre de méthode :

In [ ]:
%%php

public function insert(Request $request, ProductRepository $rep) { /* ... */ }

où l'on aura besoin :
- du service `Request` (qui représente la requête HTTP 
- et du service `ProductRepository` (qui assure la communication entre l'entité `Product`et la base de données).

### Utilisation des services

L'utilisation des services est donc transparente pour le développeur.

Chaque classe ajoutée à l'application sera automatiquement prise en charge par le conteneur de services.

## Les fournisseurs de services de Laravel

Laravel maintient, en plus du conteneur de services un ensemble de classes appelées « **fournisseurs de services** ».

Ces classes sont hébergées dans le dossier : `app/Providers`.

Ces classes sont le cœur du processus de lancement de `Laravel`. Tous les services, les vôtres ou ceux du noyau, sont initialisés par l'une d'elle. 

Ce sont elles qui vont, justement :
- constreuire le conteneur de services
- enregistrer des écouteurs d'événements
- mettre en place les classes intergicielles (_middleware_)
- déclarer les routes
- etc.

Tous les fournisseurs de services sont déclarées dans le fichier `config/app.php`, à la section `providers`.

### Pourquoi créer un fournisseur de services ?

Dans les versions récentes, il n'est pas indispensable de déclarer les services par le biais d'un fournisseur de services. Le noyau détecte automatiquement les chaînes de dépendance.

Reste donc le cas où nous avons besoin de **configurer** le service, par le biais d'uun fichier de configuration, contenu dans le dossier `config`.

Un exemple particulier est le fournisseur `RouteServiceProvider`, qui permet de configurer le processus de routage, au-delà des classes elles-mêmes.

### Créer son propre fournisseur des services

